<a href="https://colab.research.google.com/github/heinohen/Textual-Data-Analysis/blob/main/TDA_exercise11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Textual data analysis exercise 11 - basic chatbot



### TODO


* still work in progress.. finish tmrw 2025-02-20, deadline 2025-02-26

## Setup

### installs

In [ ]:
%%bash

pip3 install -q transformers torch

In [27]:
!nvidia-smi

Wed Feb 19 18:42:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   52C    P0             28W /   72W |   16791MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### imports

In [22]:
import transformers
#transformers.logging.set_verbosity_error() # no

from transformers import AutoModelForCausalLM, AutoTokenizer

## model

In [23]:
MODEL_NAME = 'HuggingFaceTB/SmolLM2-1.7B-Instruct'

In [31]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# for multiple GPUs install accelerate and do `model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")`
device = "cuda"
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

# PROGRAM

    At least 5 questions about basic facts about the world (e.g. the capital of a country)
    At least 5 arithmetic questions ranging from trivial ("what is 1+1?") to more complex
    Inform the system of a secret word (e.g. "zebra"), then after a few other questions ask it what the secret word is. Make sure you understand where the memory of that secret word is.

## data storage

In [50]:
message_history = []

## actual functionality

In [51]:
while True:
    user_input = input('Say something ("exit" to quit): ')
    if user_input == 'exit':
        break

    message_history.append({"role": "user", "content": f"{user_input}"})
    input_text=tokenizer.apply_chat_template(message_history, tokenize=False)
    inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
    outputs = model.generate(inputs,
                             max_new_tokens=50,
                             temperature=0.2,
                             top_p=0.9,
                             do_sample=True)
    print(len(inputs[0]))
    result = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True).rstrip().split()
    answer = " ".join(result[1:])
    message_history.append({"role": "assistant", "content": answer})
    print(answer)

Say something ("exit" to quit): the secret word is ZEBRA
34
Zebra is a fantastic animal! Zebras are known for their distinctive black and white stripes, which serve as camouflage in their natural habitat. They are also known for their speed and agility, making them excellent runners. Zebras are
Say something ("exit" to quit): what is ice
94
Ice is a solid form of water. It is formed when water freezes, turning from a liquid to a solid state. Ice is less dense than water, which is why it floats on top of liquid water. This unique property is
Say something ("exit" to quit): what is water
154
Water is a transparent, odorless, and tasteless liquid that is essential for life. It is composed of two hydrogen atoms and one oxygen atom, and it is the primary solvent for many biological processes. Water is vital for human
Say something ("exit" to quit): what are the three phases of water
218
The three phases of water are solid, liquid, and gas. In its solid form, water is ice. In its liquid form

# MESSAGE LOG

In [63]:
for m in message_history:
  print(m['role'], "with message =>>>>> ", m['content'])

user with message =>>>>>  the secret word is ZEBRA
assistant with message =>>>>>  Zebra is a fantastic animal! Zebras are known for their distinctive black and white stripes, which serve as camouflage in their natural habitat. They are also known for their speed and agility, making them excellent runners. Zebras are
user with message =>>>>>  what is ice
assistant with message =>>>>>  Ice is a solid form of water. It is formed when water freezes, turning from a liquid to a solid state. Ice is less dense than water, which is why it floats on top of liquid water. This unique property is
user with message =>>>>>  what is water
assistant with message =>>>>>  Water is a transparent, odorless, and tasteless liquid that is essential for life. It is composed of two hydrogen atoms and one oxygen atom, and it is the primary solvent for many biological processes. Water is vital for human
user with message =>>>>>  what are the three phases of water
assistant with message =>>>>>  The three phases of